<a href="https://colab.research.google.com/github/KeyMoney22/Plant_disease_classification-/blob/main/Flask_App_Model_Deployment%20using%20ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installation of dependencies needed
!pip install keras
!pip install tensorflow
!pip install ngrok
!pip install pyngrok
!pip install flask keras tensorflow pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.9 MB/s eta 0:00:00


In [2]:
#Importing Necessary libraries
from google.colab import drive
import os
import sys
import json
import requests
from pyngrok import ngrok
import tensorflow as tf
from flask import Flask, request, render_template, jsonify
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io
import base64

In [3]:
#Importing and mounting google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Creation of a HTML file to aid in web deployment
import os
if not os.path.exists('templates'):
    os.makedirs('templates')

html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Tomato Disease Detection</title>
</head>
<body>
    <h1>Tomato Disease Detection</h1>
    <form id="upload-form" enctype="multipart/form-data">
        <input type="file" id="file" accept="image/*">
        <button type="button" onclick="uploadImage()">Predict</button>
    </form>
    <div id="result"></div>
    <div id="image-container"></div>
    <script>
        function uploadImage() {
            var file = document.getElementById('file').files[0];
            var reader = new FileReader();
            reader.onload = function(event) {
                var data = {
                    'image': event.target.result.split(',')[1]
                };
                fetch('/predict/', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify(data)
                })
                .then(response => response.json())
                .then(data => {
                    document.getElementById('result').innerText = 'Prediction: ' + data.prediction;
                    var img = new Image();
                    img.src = 'data:image/png;base64,' + data.image;
                    img.width = 224;  // Set the desired width
                    img.height = 224;  // Set the desired height
                    var imageContainer = document.getElementById('image-container');
                    imageContainer.innerHTML = '';  // Clear any previous image
                    imageContainer.appendChild(img);
                });
            };
            reader.readAsDataURL(file);
        }
    </script>
</body>
</html>
"""

# Save the HTML content to a file
with open('templates/index.html', 'w') as file:
    file.write(html_content)

In [5]:
#Updating ngrok app authentication token for access before web deployment
!ngrok authtoken 2kAITewz0jtFcEgwKQzDndn7UsU_jWkjbLWWVtubZXFANtsM

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/Tomato_dataset/model_restune2.h5') # Make sure to include the '.h5' extension

In [7]:
# Initialize Flask app
app = Flask(__name__)

# Define a custom object dictionary
custom_objects = {
    'BatchNormalization': tf.keras.layers.BatchNormalization,
    # Add any other custom layers or objects used in your model
}

# Load the model with custom objects

model = load_model('/content/drive/MyDrive/Tomato_dataset/model_restune2.h5')

# Define class names
class_names = [
    'tomato_bacterial_spot', 'tomato_early_blight', 'tomato_late_blight',
    'tomato_leaf_mold', 'tomato_septoria_leaf_spot', 'tomato_spider_mites_two-spotted_spider_mite',
    'tomato_target_spot', 'tomato_tomato_yellow_leaf_curl_virus', 'tomato_mosaic_virus', 'tomato_healthy'
]

# Function to decode and preprocess the image
def preprocess_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Route for the main page
@app.route('/')
def index():
    return render_template('index.html')  # Ensure you have an index.html file in the templates folder

# Route for prediction
@app.route('/predict/', methods=['POST'])
def predict():
    message = request.get_json(force=True)
    encoded_image = message['image']
    decoded_image = base64.b64decode(encoded_image)
    image = Image.open(io.BytesIO(decoded_image))

    # Preprocess the image
    processed_image = preprocess_image(image, target_size=(224, 224))  # Update target_size if needed

    # Perform prediction
    predictions = model.predict(processed_image)
    predicted_class = class_names[np.argmax(predictions)]

    response = {
        'prediction': predicted_class,
        'image': message['image']
    }
    return jsonify(response)

# Run the app
if __name__ == "__main__":
    # Set up ngrok tunnel
    public_url = ngrok.connect(addr='5000')
    print(" * ngrok tunnel URL:", public_url)

    app.run(host='0.0.0.0', port=5000)

 * ngrok tunnel URL: NgrokTunnel: "https://e7f8-34-19-112-12.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 18:56:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 18:56:18] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 18:57:09] "POST /predict/ HTTP/1.1" 200 -


To access the web-app, we use the printed URL link for NgrokTunnel above.